## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):


### Confidence Intervals:
1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?


### Chi-squared tests:
4. Take a dataset that we have used in the past in class that has **categorical** variables. Pick two of those categorical variables and run a chi-squared tests on that data
  - By hand using Numpy
  - In a single line using Scipy


In [0]:
# Import external libraries:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_1samp, t, norm
import scipy.stats as stats

# Load in csv file of house votes from '84:
house_votes = pd.read_csv('house-votes-84.data', header=None, na_values='?')

# Check to see if the data loades with the frist 5 rows:
# display(house_votes.head())

# View the shape of our data to see if it matches the original data repo:
# print(house_votes.shape)

# Open the names document to get the attribute names
with open('house-votes-84.names','r') as file:
    names_file = file.read()
    # print(names_file)

# Create a list of Attribute names to use as columns:
cols = ['party','handicap-infants','water-project-cost-sharing',
        'adoption-of-the-budget-resolution','physician-fee-freeze',
        'el-salvador-aid','religious-groups-in-schools',
        'anti-satellite-test-ban','aid-to-nicaraguan-contras','mx-missile',
        'immigration','synfuels-corporation','education-spending',
        'superfund-right-to-sue','crime','duty-free-exports',
        'export-administration-act-south-africa']

# Apply cols list to our data frame:
house_votes.columns = cols

# View the first five rows to confirm the change was saved:
# house_votes.head()

# Here we will map our yes and no values:
house_votes = house_votes.replace({'y':1,'n':0})

# Now I want to create a group for republicans and a group ro democrats:
republicans = house_votes[house_votes['party']=='republican']
democrats = house_votes[house_votes['party']=='democrat']

In [53]:
house_votes[['party','religious-groups-in-schools']].describe()

,religious-groups-in-schools
count,424.000000
mean,0.641509
std,0.480124
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [41]:
# get the mean of both republicans and democrats.
print(republicans['religious-groups-in-schools'].mean())
print(democrats['religious-groups-in-schools'].mean())

0.8975903614457831
0.47674418604651164


In [42]:
ttest_1samp(republicans['religious-groups-in-schools'],
            house_votes['religious-groups-in-schools'].mean(),nan_policy='omit')

Ttest_1sampResult(statistic=10.84949937999739, pvalue=4.850660006171671e-21)

In [45]:
# is there a significant difference in votes for religious groups in schools  among republicans 
results = ttest_ind(republicans['religious-groups-in-schools'],democrats['religious-groups-in-schools'],nan_policy='omit')
results

Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)

In [47]:
# useing a crosstab to further examine data
rep_dem_x = pd.crosstab(house_votes['party'],house_votes['religious-groups-in-schools'],normalize='index')
rep_dem_x

religious-groups-in-schools,0.0,1.0
party,,
democrat,0.523256,0.476744
republican,0.102410,0.897590


In [48]:
rep_dem_x.describe()

religious-groups-in-schools,0.0,1.0
count,2.000000,2.000000
mean,0.312833,0.687167
std,0.297583,0.297583
min,0.102410,0.476744
25%,0.207621,0.581956
50%,0.312833,0.687167
75%,0.418044,0.792379
max,0.523256,0.897590


## 1. **Generate and numericaly represent a confidence interval:**

In [0]:
# Step 1. We need to define our values:
# s = 'std of sample'
# n = 'sample size'
# t = 'test statistic'
# x_bar = 'sample mean'

>  <b>Confidence Interval Formula:</b>
$\bar{X}\pm{t}\frac{S}{\sqrt{n}}$

In [0]:
def confidence_interval(data,confidence=0.95):
    data = np.array(data) # convert data to array
    x_bar = np.mean(data) # get the mean of the data
    n = len(data) # length of the sample
    std_err = np.std(data, ddof=1)/np.sqrt(n) # get the standard error
    moe = std_err * stats.t.ppf((1 + confidence) / 2.0, n - 1) 
    return (x_bar, x_bar - moe, x_bar + moe)

In [0]:
# Function to remove nans from columns
def removeNan(df,col_name=[]):
    '''Function to remove nans from dataframe columns'''
    col = df[col_name]
    np.isnan(col)
    return col[~np.isnan(col)]

In [77]:
house_votes = removeNan(house_votes,['religious-groups-in-schools'])
house_votes['religious-groups-in-schools']                                                       

KeyError: ignored

In [64]:
confidence_interval(house_votes['religious-groups-in-schools'])

(nan, nan, nan)

## Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.
3. Refactor your code so it is elegant, readable, and can be easily run for all issues.

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)